# Постановка задачи
Итак, давайте применим новые знания для решения следующей задачи: имеется список отзывов о ресторанах. Необходимо разделить их на положительные и отрицательные. В случае невозможности классификации ставить 'undef'.

Файл с отзывами texts_opinions.txt

Файл с отзывами имеет кодировку UTF-8 и может некорректно отображаться при открытии в браузере.

Для классификации для каждого отзыва будем использовать следующий алгоритм:

1. Предварительно составляем список основ слов, которые характеризуют положительные и отрицательные отзывы. Используем SnowballStemmer библиотеки NLTK (можно было использовать Pymystem, сейчас для простоты используем NLTK).
2. Разбиваем текст отзыва на отдельные слова (здесь нам заранее придется удалить все знаки препинания).
3. Заменяем каждое слово на его основу с помощью SnowballStemmer библиотеки NLTK.
4. Ищем основу каждого слова отзыва среди основ слов из пункта 1. Считаем каких слов получилось больше - из списка "положительных" или "отрицательных" слов. Если таких слов в отзыве не нашлось или их число совпало, то возвращаем 'undef'.

После классификации отзывов мы можем сверить наш результат с "правильной" классификацией texts_ratings.txt, которую выставляли сами пользователи, когда писали эти отзывы.

Файл texts_ratings.txt

Несколько замечаний:

1. Сейчас мы не рассматриваем качество и правдивость составленных отзывов. Наша задача - научиться пользоваться инструментами для подобных задач.
2. Существует множество алгоритмов решения этой задачи, в том числе с помощью машинного обучения. Абсолютное большинство из них начинаются также как и наш - со стемминга или лемматизации исходных текстов.
3. Аналогичный подход может сильно помочь вам в задачах фильтрации данных по словам.

# Разбивка на слова
Итак, сначала составим список основ слов, которые характерны для положительных и отрицательных отзывов. Чтобы не учитывать их многочисленные формы используем стеммер NLTK. Например, найдем основу слова "благодарны":

In [36]:
from nltk.stem import SnowballStemmer
from yaml import load
import re
import os

In [4]:
snowball_stemmer = SnowballStemmer("russian")
print('1', snowball_stemmer.stem('благодарны'))

for word in ['благодарность', 'благодарностью', 'благодарны']:
    print(snowball_stemmer.stem( word ))

1 благодарн
благодарн
благодарн
благодарн


In [8]:
with open('./module16_files/params.yaml', encoding='utf-8') as f:
    params = load(f)

Сначала необходимо удалить из текста знаки препинания, чтобы его можно легко было разбить на слова через пробел. Воспользуемся методом translate. Метод берет список знаков препинания symbols и применяет к ним метод translate. При этом заменяя их на пробелы. Для этого мы заводим строку spaces из такого же количества пробелов, что и symbols:

In [23]:
def clear_punctuation(text):
    """Удаление знаком пунктуации из текста text"""

    return re.sub(r'[^\w\s]', '', text)

In [25]:
text = 'Просто шикарный клуб! Ходили с другом на "Animal Джаz"! Остались очень довольны, атмосфера очень уютная, дружелюбная, есть второй этаж, бар'

In [26]:
clear_punctuation(text)

'Просто шикарный клуб Ходили с другом на Animal Джаz Остались очень довольны атмосфера очень уютная дружелюбная есть второй этаж бар'

# Построение классификатора
Чтобы получить список слов, основы которых входят в список "положительных" наборов слов, достаточно использовать list comprehension:

In [28]:
text_no_punctuation = clear_punctuation(text)

positive_words_list = [x for x in text_no_punctuation.split(' ') 
                       if snowball_stemmer.stem(x) in params['positive']]

In [30]:
# В этом тексте оказалось только одно "положительное" слово. 
# Есть чем расширить наши списки для улучшения модели классификации:

print(positive_words_list)
# ['довольны']

# Количество слов в таком списке считаем с помощью функции len:

positive_words_count = len(positive_words_list)

positive_words_count

['довольны']


1

In [31]:
def classifier(text):

    """Классификация отзыва text на 'positive', 'negative' и 'undef' по совпадающим основам слов из params.yaml"""

    text = clear_punctuation(text)
    positive_words_count = len([x for x in text.split() if snowball_stemmer.stem(x) in params['positive']])
    negative_words_count = len([x for x in text.split() if snowball_stemmer.stem(x) in params['negative']])
    if positive_words_count > negative_words_count:
        return 'positive'
    elif positive_words_count < negative_words_count:
        return 'negative'
    return 'undef'

Проверяем как работает (на очевидных примерах):

In [32]:
classifier(text)

'positive'

In [33]:
text2 = 'Ужасное место. Сотрудники клуба от них в восторге! А культурным людям тут не место.'

classifier(text2)

'negative'

Прогоним наш файл с отзывами через классификатор. В этом скрипте мы используем одни и те же файлы для чтения и записи. В таких случаях рекомендуется "закрывать" их с помощью метода close, чтобы при следующем открытии не возникало проблем и каждый раз не придумывать новые названия переменных:

In [37]:
path = './module16_files/'

f = open(os.path.join(path, 'texts_opinions.txt'), encoding = 'utf-8')
f_classified = open(os.path.join(path, 'texts_classified.txt'), 'w', encoding = 'utf-8')

for line in f:
    line = line.strip()    
    f_classified.write('{}\n'.format(classifier(line)))

f.close()
f_classified.close()

# Проверка точности модели
Теперь давайте сравним оценки нашего классификатора с реальными оценками пользователей в файле texts_ratings.txt. Будем пользовать следующими правилами:

1. Не учитываем отзывы, которые мы не смогли классифицировать (значение 'undef').
2. Для определенных типов отзывов (positive и negative) считаем их общее количество в переменной total_defined_ratings.
3. Если оценка отзыва в файлах совпала, то увеличиваем переменную right_classifications на 1.
4. В конце алгоритма выводим долю верно угаданных отзывов.

In [39]:
f_classified = open(os.path.join(path, 'texts_classified.txt'), encoding = 'utf-8')
f_ratings = open(os.path.join(path, 'texts_ratings.txt'), encoding = 'utf-8')

classified_list = [line.strip() for line in f_classified]
ratings_list = [line.strip() for line in f_ratings]
right_classifications = 0
total_defined_ratings = 0

for i in range(len(classified_list)):
    if classified_list[i] != 'undef':
        total_defined_ratings += 1
        if classified_list[i] == ratings_list[i]:
            right_classifications += 1

print('Доля верно классифицированных отзывов: {:.0%}'.format(right_classifications / total_defined_ratings))

f_classified.close()
f_ratings.close()

Доля верно классифицированных отзывов: 77%


Итак, при первой самой простой модели мы получили точность классификации 77%. Конечно, это без учета того, что часть отзыва мы не классифицировали. Но долю неопределенных отзывов можно уменьшить, добавив в наш словарь больше "положительных" и "отрицательных" слов.

Попробуйте улучшить классификатор, добавив в params.yaml больше слов.